# Data Preparation
## Import libararies

In [26]:
from langdetect import detect, detect_langs, DetectorFactory
from dotenv import load_dotenv
import pandas as pd
import numpy as np
import openai
import json
import sys
import re
import os

from rapidfuzz import fuzz, utils
from collections import Counter
from itertools import chain


src_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'src'))
if src_path not in sys.path:
    sys.path.insert(0, src_path)

%load_ext autoreload
%autoreload 2

from jobs_tools import data_cleaning, chat_gpt, tests_helpers


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
load_dotenv()
client = openai.OpenAI()
client_async = openai.AsyncOpenAI()

## Initial data cleaning

In [16]:
jobs_df = pd.read_csv('../data/csv/jobs_data.csv').copy()

# Step 1
jobs_df = data_cleaning.remove_exact_duplicates(jobs_df)
# Step 2
jobs_df_clean = data_cleaning.remove_job_id_duplicates(jobs_df)

jobs_df_clean.head()

Step 1: Removing exact duplicates
- Initial number of rows: 5834
- Duplicates removed: 0
- Remaining rows: 5834

Step 2: Removing Job ID duplicates within the same country
- Initial row count: 5834
- Duplicates removed: 2971
- Remaining rows: 2863



,Location,Region,EU Member,Schengen Agreement,Google Domain Type,Google Domain Used,Job Title,Company Name,Job Location,Apply Options,Job Description,Work from home,Salary,Schedule type,Qualifications,Job ID,Search Date,Search Query
0,Austria,Europe,True,True,local,google.at,Sandbox Innovation Sdn. Bhd. | Senior Mobile A...,Tideri Jobbörse,Austria,"Trabajo.org - Stellenangebote, Arbeit",We are a dynamic FinTech company headquartered...,NaN,NaN,Full-time,NaN,eyJqb2JfdGl0bGUiOiJTYW5kYm94IElubm92YXRpb24gU2...,2025-01-13 12:33:21 UTC,iOS developer
1,Austria,Europe,True,True,default,google.com,Senior Ios Developer,Pyramid Global Technologies,Austria,"Trabajo.org - Stellenangebote, Arbeit, StudySm...",A minimum of 6+ years of concurrent commercial...,NaN,NaN,Full-time,No degree mentioned,eyJqb2JfdGl0bGUiOiJTZW5pb3IgSW9zIERldmVsb3Blci...,2025-01-13 12:20:43 UTC,iOS developer
2,Austria,Europe,True,True,default,google.com,iOS Developer - Permanent remote,Bluestorm Recruitment by Dazzle,Austria,Jooble,iOS Developer\r\n\r\nOur client is a leading m...,NaN,NaN,Full-time,No degree mentioned,eyJqb2JfdGl0bGUiOiJpT1MgRGV2ZWxvcGVyIC0gUGVybW...,2025-01-13 12:20:43 UTC,iOS developer
3,Austria,Europe,True,True,default,google.com,Sr. Ios Developer,Bykon,Austria,"Trabajo.org - Stellenangebote, Arbeit",In ByKon we're looking for an exceptional Sr.\...,NaN,NaN,Full-time,NaN,eyJqb2JfdGl0bGUiOiJTci4gSW9zIERldmVsb3BlciIsIm...,2025-01-13 12:20:43 UTC,iOS developer
4,Austria,Europe,True,True,default,google.com,Software Engineer/ iOS,Bitpanda,Anywhere,GrabJobs,Who we are\r\n\r\nWe simplify wealth creation....,True,NaN,Full-time,No degree mentioned,eyJqb2JfdGl0bGUiOiJTb2Z0d2FyZSBFbmdpbmVlci8gaU...,2025-01-13 12:20:43 UTC,iOS developer


## Detect the language of job postings using the `langdetect` library

In [17]:
DetectorFactory.seed = 0  # We record the result so that there are no accidental changes

def detect_language(text):
    try:
        return detect(text)
    except:
        return "unknown"
    
def detect_language_with_confidence(text):
    try:
        lang_probs = detect_langs(text)
        if lang_probs:
            return str(lang_probs[0])  # Format: 'en:0.99'
    except:
        return "unknown"

jobs_df_clean["Language langdetect confidence"] = jobs_df_clean["Job Description"].apply(detect_language_with_confidence)
jobs_df_clean.to_csv('../data/csv/jobs_data_langdetect.csv', index=False)

#jobs_df_clean = pd.read_csv('../data/csv/jobs_data_langdetect.csv').copy()
jobs_df_clean.head()

,Location,Region,EU Member,Schengen Agreement,Google Domain Type,Google Domain Used,Job Title,Company Name,Job Location,Apply Options,Job Description,Work from home,Salary,Schedule type,Qualifications,Job ID,Search Date,Search Query,Language langdetect confidence
0,Austria,Europe,True,True,local,google.at,Sandbox Innovation Sdn. Bhd. | Senior Mobile A...,Tideri Jobbörse,Austria,"Trabajo.org - Stellenangebote, Arbeit",We are a dynamic FinTech company headquartered...,NaN,NaN,Full-time,NaN,eyJqb2JfdGl0bGUiOiJTYW5kYm94IElubm92YXRpb24gU2...,2025-01-13 12:33:21 UTC,iOS developer,en:0.9999973661975282
1,Austria,Europe,True,True,default,google.com,Senior Ios Developer,Pyramid Global Technologies,Austria,"Trabajo.org - Stellenangebote, Arbeit, StudySm...",A minimum of 6+ years of concurrent commercial...,NaN,NaN,Full-time,No degree mentioned,eyJqb2JfdGl0bGUiOiJTZW5pb3IgSW9zIERldmVsb3Blci...,2025-01-13 12:20:43 UTC,iOS developer,en:0.999996472315817
2,Austria,Europe,True,True,default,google.com,iOS Developer - Permanent remote,Bluestorm Recruitment by Dazzle,Austria,Jooble,iOS Developer\r\n\r\nOur client is a leading m...,NaN,NaN,Full-time,No degree mentioned,eyJqb2JfdGl0bGUiOiJpT1MgRGV2ZWxvcGVyIC0gUGVybW...,2025-01-13 12:20:43 UTC,iOS developer,en:0.9999944159931299
3,Austria,Europe,True,True,default,google.com,Sr. Ios Developer,Bykon,Austria,"Trabajo.org - Stellenangebote, Arbeit",In ByKon we're looking for an exceptional Sr.\...,NaN,NaN,Full-time,NaN,eyJqb2JfdGl0bGUiOiJTci4gSW9zIERldmVsb3BlciIsIm...,2025-01-13 12:20:43 UTC,iOS developer,en:0.9999966313140446
4,Austria,Europe,True,True,default,google.com,Software Engineer/ iOS,Bitpanda,Anywhere,GrabJobs,Who we are\r\n\r\nWe simplify wealth creation....,True,NaN,Full-time,No degree mentioned,eyJqb2JfdGl0bGUiOiJTb2Z0d2FyZSBFbmdpbmVlci8gaU...,2025-01-13 12:20:43 UTC,iOS developer,en:0.9999964386438764


### Split the column with "en:0.99" into "en" and "0.99"

In [18]:
jobs_df_clean = data_cleaning.split_values(jobs_df_clean)
jobs_df_clean.head()

,Location,Region,EU Member,Schengen Agreement,Google Domain Type,Google Domain Used,Job Title,Company Name,Job Location,Apply Options,Job Description,Work from home,Salary,Schedule type,Qualifications,Job ID,Search Date,Search Query,Language,Confidence
0,Austria,Europe,True,True,local,google.at,Sandbox Innovation Sdn. Bhd. | Senior Mobile A...,Tideri Jobbörse,Austria,"Trabajo.org - Stellenangebote, Arbeit",We are a dynamic FinTech company headquartered...,NaN,NaN,Full-time,NaN,eyJqb2JfdGl0bGUiOiJTYW5kYm94IElubm92YXRpb24gU2...,2025-01-13 12:33:21 UTC,iOS developer,en,0.999997
1,Austria,Europe,True,True,default,google.com,Senior Ios Developer,Pyramid Global Technologies,Austria,"Trabajo.org - Stellenangebote, Arbeit, StudySm...",A minimum of 6+ years of concurrent commercial...,NaN,NaN,Full-time,No degree mentioned,eyJqb2JfdGl0bGUiOiJTZW5pb3IgSW9zIERldmVsb3Blci...,2025-01-13 12:20:43 UTC,iOS developer,en,0.999996
2,Austria,Europe,True,True,default,google.com,iOS Developer - Permanent remote,Bluestorm Recruitment by Dazzle,Austria,Jooble,iOS Developer\r\n\r\nOur client is a leading m...,NaN,NaN,Full-time,No degree mentioned,eyJqb2JfdGl0bGUiOiJpT1MgRGV2ZWxvcGVyIC0gUGVybW...,2025-01-13 12:20:43 UTC,iOS developer,en,0.999994
3,Austria,Europe,True,True,default,google.com,Sr. Ios Developer,Bykon,Austria,"Trabajo.org - Stellenangebote, Arbeit",In ByKon we're looking for an exceptional Sr.\...,NaN,NaN,Full-time,NaN,eyJqb2JfdGl0bGUiOiJTci4gSW9zIERldmVsb3BlciIsIm...,2025-01-13 12:20:43 UTC,iOS developer,en,0.999997
4,Austria,Europe,True,True,default,google.com,Software Engineer/ iOS,Bitpanda,Anywhere,GrabJobs,Who we are\r\n\r\nWe simplify wealth creation....,True,NaN,Full-time,No degree mentioned,eyJqb2JfdGl0bGUiOiJTb2Z0d2FyZSBFbmdpbmVlci8gaU...,2025-01-13 12:20:43 UTC,iOS developer,en,0.999996


### Handling rows with confidence below 0.99

In [19]:
low_confidence_df = data_cleaning.low_confidence_data(jobs_df_clean)
jobs_df_clean = jobs_df_clean.drop(low_confidence_df.index)

#### ChatGPT detects language

In [20]:
df_result = await chat_gpt.chatgpt_async(
    input_column_name="Job Description", 
    output_column_name="Language gpt-4o-2024-11-20",
    input_text_length=None,
    output_text_length=1,
    num_rows=None,  
    df=low_confidence_df.copy(), 
    user_prompt="Detect the language of the text and return ONLY the ISO country code (e.g., en, fr, de, ect.). Text:",
    gpt_model="gpt-4o-2024-11-20",
    client=client_async,
    batch_size=10,
    concurrency_limit=10,
    cache_file="../data/cache/language_cache_gpt-4o-2024-11-20.json"
)
df_result.head()

Processing Batches: 100%|██████████| 2/2 [00:00<00:00, 60.77it/s]


,Location,Region,EU Member,Schengen Agreement,Google Domain Type,Google Domain Used,Job Title,Company Name,Job Location,Apply Options,...,Work from home,Salary,Schedule type,Qualifications,Job ID,Search Date,Search Query,Language,Confidence,Language gpt-4o-2024-11-20
1614,Poland,Europe,True,True,default,google.com,Senior Android Developer,FREAM S. A.,"Wrocław, Poland","The:Protocol, No Fluff Jobs, Capgemini Careers...",...,NaN,NaN,Contractor,No degree mentioned,eyJqb2JfdGl0bGUiOiJTZW5pb3IgQW5kcm9pZCBEZXZlbG...,2025-01-13 12:11:12 UTC,Android developer,nl,0.571425,en
528,Czechia,Europe,True,True,default,google.com,Android Developer (part-time: 2h/week) @ Exper...,Experis Polska,Anywhere,"Jooble, Jobs Trabajo.org",...,True,NaN,Full-time,No degree mentioned,eyJqb2JfdGl0bGUiOiJBbmRyb2lkIERldmVsb3BlciAocG...,2025-01-13 12:09:27 UTC,Android developer,en,0.571427,cs
434,Cyprus,Europe,False,False,default,google.com,iOS Applications Developer,MetaQuotes,"Limassol, Cyprus","MetaQuotes, Evoplay, Lever, Хабр Карьера - Hab...",...,NaN,NaN,Full-time,No degree mentioned,eyJqb2JfdGl0bGUiOiJpT1MgQXBwbGljYXRpb25zIERldm...,2025-01-13 12:21:15 UTC,iOS developer,it,0.714282,en
780,Germany,Europe,True,True,default,google.com,JUNIOR IOS DEVELOPER,Check24,Germany,Layboard,...,NaN,€4.5K a month,Full-time,No degree mentioned,eyJqb2JfdGl0bGUiOiJKVU5JT1IgSU9TIERFVkVMT1BFUi...,2025-01-13 12:22:03 UTC,iOS developer,en,0.714283,ru
481,Czechia,Europe,True,True,default,google.com,iOS developer - Ostrava,confidential,"Ostrava, Czechia",Jooble,...,NaN,NaN,Full-time,No degree mentioned,eyJqb2JfdGl0bGUiOiJpT1MgZGV2ZWxvcGVyIC0gT3N0cm...,2025-01-13 12:21:22 UTC,iOS developer,no,0.714283,cs


#### Manual check

In [21]:
df_result['Manual check'] = ["drop", "cs", "drop", "ru", "drop", "fr", "en", "pl", "en", "drop", "fr", "en", "pl", "sv", "zh", "fr"]
df_result['Drop'] = [True, False, True, False, True, False, False, False, False, True, False, False, False, False, False, False]
df_result = df_result.loc[~df_result['Drop']]

df_result = data_cleaning.clean_data(df_result)
df_result.head()

,Location,Region,EU Member,Schengen Agreement,Google Domain Type,Google Domain Used,Job Title,Company Name,Job Location,Apply Options,Job Description,Work from home,Salary,Schedule type,Qualifications,Job ID,Search Date,Search Query,Language,Confidence
528,Czechia,Europe,True,True,default,google.com,Android Developer (part-time: 2h/week) @ Exper...,Experis Polska,Anywhere,"Jooble, Jobs Trabajo.org",O pozici / o projektu\r\nPůvodní popisek. Andr...,True,NaN,Full-time,No degree mentioned,eyJqb2JfdGl0bGUiOiJBbmRyb2lkIERldmVsb3BlciAocG...,2025-01-13 12:09:27 UTC,Android developer,cs,0.571427
780,Germany,Europe,True,True,default,google.com,JUNIOR IOS DEVELOPER,Check24,Germany,Layboard,Требования 1-2 years of experience with iOS de...,NaN,€4.5K a month,Full-time,No degree mentioned,eyJqb2JfdGl0bGUiOiJKVU5JT1IgSU9TIERFVkVMT1BFUi...,2025-01-13 12:22:03 UTC,iOS developer,ru,0.714283
325,Canada,Northern America,False,False,default,google.com,"Développeur senior, Android / Senior Android D...",Cerence,"Montreal, Quebec, Canada","Indeed, Built In, Eluta.ca, Glassdoor, Adzuna,...",A Moving Experience.\r\n\r\n(English version b...,NaN,NaN,Full-time,NaN,eyJqb2JfdGl0bGUiOiJEw6l2ZWxvcHBldXIgc2VuaW9yLC...,2025-01-13 12:09:10 UTC,Android developer,fr,0.714285
2105,Sweden,Europe,True,True,default,google.com,Konsultuppdrag Ios and Android Developers - Of...,Senterprise,ستوكهولم، السويد,Emprego.pt,To one of our clients we are now looking for 1...,NaN,NaN,دوام كامل,NaN,eyJqb2JfdGl0bGUiOiJLb25zdWx0dXBwZHJhZyBJb3MgYW...,2025-01-13 12:11:47 UTC,Android developer,en,0.857138
1591,Poland,Europe,True,True,default,google.com,Android Developer,ALAN Systems,"Silesian Voivodeship, Poland",JobLeads,"Cześć odkrywco kodu!\r\n\r\nCzy jesteś gotowy,...",NaN,PLN 180K–PLN 240K a year,Full-time,No degree mentioned,eyJqb2JfdGl0bGUiOiJBbmRyb2lkIERldmVsb3BlciIsIm...,2025-01-13 12:11:12 UTC,Android developer,pl,0.857139


#### Merge back

In [22]:
jobs_df_clean = pd.concat([jobs_df_clean, df_result], ignore_index=True)
jobs_df_clean = jobs_df_clean.sort_values(['Location'])
jobs_df_clean.reset_index(inplace=True, drop=True)
jobs_df_clean = jobs_df_clean.drop(columns=['Confidence'])

jobs_df_clean.head()

,Location,Region,EU Member,Schengen Agreement,Google Domain Type,Google Domain Used,Job Title,Company Name,Job Location,Apply Options,Job Description,Work from home,Salary,Schedule type,Qualifications,Job ID,Search Date,Search Query,Language
0,Austria,Europe,True,True,local,google.at,Sandbox Innovation Sdn. Bhd. | Senior Mobile A...,Tideri Jobbörse,Austria,"Trabajo.org - Stellenangebote, Arbeit",We are a dynamic FinTech company headquartered...,NaN,NaN,Full-time,NaN,eyJqb2JfdGl0bGUiOiJTYW5kYm94IElubm92YXRpb24gU2...,2025-01-13 12:33:21 UTC,iOS developer,en
1,Austria,Europe,True,True,default,google.com,Mobile Application Developer,Pearson Carter,Austria,"Trabajo.org - Stellenangebote, Arbeit",Lead Mobile Developer | Hyper Growth Startup |...,NaN,NaN,Full-time,No degree mentioned,eyJqb2JfdGl0bGUiOiJNb2JpbGUgQXBwbGljYXRpb24gRG...,2025-01-13 12:08:51 UTC,Android developer,en
2,Austria,Europe,True,True,default,google.com,"Android Developer – Kotlin (Austria based, Hyb...",Bitcoin Devs Company,"Vienna, Austria",Jobs3,Overview:\r\nThe Android Developer – Kotlin po...,NaN,NaN,Contractor,No degree mentioned,eyJqb2JfdGl0bGUiOiJBbmRyb2lkIERldmVsb3BlciDigJ...,2025-01-13 12:08:48 UTC,Android developer,en
3,Austria,Europe,True,True,default,google.com,Android & iOS Developer,ventopay gmbh,Austria,StudySmarter - Talents,Was sind deine Aufgaben?\r\n• Du gestaltest at...,NaN,NaN,Full-time,No degree mentioned,eyJqb2JfdGl0bGUiOiJBbmRyb2lkIFx1MDAyNiBpT1MgRG...,2025-01-13 12:20:43 UTC,iOS developer,de
4,Austria,Europe,True,True,default,google.com,iOS Developer Up3 (f/m/d),Drei Österreich,"Vienna, Austria","MyAbility.jobs, Drei., Jobted.at",Do you want to push the frontier of digital se...,NaN,NaN,Full-time,No degree mentioned,eyJqb2JfdGl0bGUiOiJpT1MgRGV2ZWxvcGVyIFVwMyAoZi...,2025-01-13 12:20:39 UTC,iOS developer,en


## Translate job descriptions into english 

Original:
This is a [SRC] to [TGT]
translation, please provide
the [TGT] translation for these
sentences:


My version:
This is a [SRC] to [TGT]
translation, please provide
the [TGT] translation for this
job description:

In [23]:
with open('../data/json/language_map.json', 'r') as f:
    language_mapping = json.load(f)

df_translated = await chat_gpt.translate_non_english_descriptions(
    df=jobs_df_clean.copy(),
    language_col="Language",
    job_desc_col="Job Description",
    translated_col="Job Description English",
    client=client_async,
    gpt_model="gpt-4o-2024-11-20",
    batch_size=10,
    concurrency_limit=10,
    language_mapping=language_mapping,
    cache_file="../data/cache/job_description_english_cache_V2_gpt-4o-2024-11-20.json"
)

Processing Batches: 100%|██████████| 42/42 [00:01<00:00, 39.25it/s]


### Drop too short job descriptions

In [ ]:
df_translated = df_translated[df_translated['Job Description English'].str.split().str.len() >= 14].reset_index(drop=True)
#df_translated.to_csv('../data/csv/df_translated.csv', index=False)

## Split job descriptions into structured sections

### Comparison of ChatGPT's Extraction vs. Ground Truth

Average Token Set Ratio per section:
- Platform: 98.33
- Salary: 97.35
- Requirements: 98.61
- Nice to have: 93.38
- Responsibilities: 90.88
- Benefits: 91.85
- Overall Average Token Set Ratio: 95.07

Tests: [link](./tests.ipynb#Split-job-descriptions-into-structured-sections)

### Full Extraction 

In [25]:
system_prompt = """
You are an AI assistant. Your role is to extract specific information from job descriptions and format them in a strict structure.
"""

user_prompt = """
I will provide a job description. Please extract and present the information in **this exact order**:

1. Platform: (Android/iOS/Cross-platform)
2. Salary: (If stated; otherwise "Not mentioned")
3. Requirements: (verbatim from the job description or "Not mentioned")
4. Nice to have: (verbatim or "Not mentioned")
5. Responsibilities: (verbatim or "Not mentioned")
6. Benefits: (verbatim or "Not mentioned")

**Guidelines**:
- **DO NOT reword, paraphrase, or summarize** any part of the job description. Copy the sentences exactly as they appear.
- Combine all mandatory or required skill sections (e.g., "Requirements," "Skills," "Key Technologies," "About You") under **Requirements**.
- If the job description specifically says something is "a plus," "beneficial," or otherwise indicates it’s optional, place it under "Nice to have" even if it appears under a "Requirements" heading in the job description.
- If there is an "About you" or "About Role" section (or similar) that describs duties or tasks, include those under "Responsibilities".
- If the information is not in the job description, write "Not mentioned" for that section.
- For multiple platforms (e.g., Android, iOS), list them all in **Platform** and use headings under Requirements (and other sections, if needed) like "General Requirements:", "For Android Developers:", "For iOS Developers:".
- Present your answer **only** in the format above.

---
Here is the job description:
"""

df_extracted = await chat_gpt.chatgpt_async(
    input_column_name="Job Description English", 
    output_column_name="Job Description Extracted",
    input_text_length=None,
    output_text_length=None,
    num_rows=None,  
    df=df_translated.copy(), 
    system_prompt=system_prompt,
    user_prompt=user_prompt,
    gpt_model="gpt-4o-2024-11-20",
    client=client_async,
    batch_size=30,
    concurrency_limit=35,
    cache_file="../data/cache/job_description_extracted_cache_gpt-4o-2024-11-20.json"
)
df_extracted.head()

Processing Batches: 100%|██████████| 95/95 [00:11<00:00,  8.09it/s]


,Location,Region,EU Member,Schengen Agreement,Google Domain Type,Google Domain Used,Job Title,Company Name,Job Location,Apply Options,...,Work from home,Salary,Schedule type,Qualifications,Job ID,Search Date,Search Query,Language,Job Description English,Job Description Extracted
0,Austria,Europe,True,True,local,google.at,Sandbox Innovation Sdn. Bhd. | Senior Mobile A...,Tideri Jobbörse,Austria,"Trabajo.org - Stellenangebote, Arbeit",...,NaN,NaN,Full-time,NaN,eyJqb2JfdGl0bGUiOiJTYW5kYm94IElubm92YXRpb24gU2...,2025-01-13 12:33:21 UTC,iOS developer,en,We are a dynamic FinTech company headquartered...,"1. Platform: Android, iOS, Cross-platform \n2..."
1,Austria,Europe,True,True,default,google.com,Mobile Application Developer,Pearson Carter,Austria,"Trabajo.org - Stellenangebote, Arbeit",...,NaN,NaN,Full-time,No degree mentioned,eyJqb2JfdGl0bGUiOiJNb2JpbGUgQXBwbGljYXRpb24gRG...,2025-01-13 12:08:51 UTC,Android developer,en,Lead Mobile Developer | Hyper Growth Startup |...,1. Platform: Cross-platform \n2. Salary: $160...
2,Austria,Europe,True,True,default,google.com,"Android Developer – Kotlin (Austria based, Hyb...",Bitcoin Devs Company,"Vienna, Austria",Jobs3,...,NaN,NaN,Contractor,No degree mentioned,eyJqb2JfdGl0bGUiOiJBbmRyb2lkIERldmVsb3BlciDigJ...,2025-01-13 12:08:48 UTC,Android developer,en,Overview:\r\nThe Android Developer – Kotlin po...,1. Platform: Android \n2. Salary: Not mention...
3,Austria,Europe,True,True,default,google.com,Android & iOS Developer,ventopay gmbh,Austria,StudySmarter - Talents,...,NaN,NaN,Full-time,No degree mentioned,eyJqb2JfdGl0bGUiOiJBbmRyb2lkIFx1MDAyNiBpT1MgRG...,2025-01-13 12:20:43 UTC,iOS developer,de,**What are your tasks?** \n- You design attra...,1. Platform: Android/iOS \n2. Salary: Not men...
4,Austria,Europe,True,True,default,google.com,iOS Developer Up3 (f/m/d),Drei Österreich,"Vienna, Austria","MyAbility.jobs, Drei., Jobted.at",...,NaN,NaN,Full-time,No degree mentioned,eyJqb2JfdGl0bGUiOiJpT1MgRGV2ZWxvcGVyIFVwMyAoZi...,2025-01-13 12:20:39 UTC,iOS developer,en,Do you want to push the frontier of digital se...,1. Platform: iOS \n2. Salary: The gross annua...


## Extracting sections into columns

In [27]:
df_sections = df_extracted.copy()

df_sections["Platform"] = df_sections["Job Description Extracted"].apply(lambda x: tests_helpers.extract_section(x, r"1\. Platform"))
df_sections["Salary_E"] = df_sections["Job Description Extracted"].apply(lambda x: tests_helpers.extract_section(x, r"2\. Salary"))
df_sections["Requirements"] = df_sections["Job Description Extracted"].apply(lambda x: tests_helpers.extract_section(x, r"3\. Requirements"))
df_sections["Nice to have"] = df_sections["Job Description Extracted"].apply(lambda x: tests_helpers.extract_section(x, r"4\. Nice to have"))
df_sections["Responsibilities"] = df_sections["Job Description Extracted"].apply(lambda x: tests_helpers.extract_section(x, r"5\. Responsibilities"))
df_sections["Benefits"] = df_sections["Job Description Extracted"].apply(lambda x: tests_helpers.extract_section(x, r"6\. Benefits"))

df_sections.head()

,Location,Region,EU Member,Schengen Agreement,Google Domain Type,Google Domain Used,Job Title,Company Name,Job Location,Apply Options,...,Search Query,Language,Job Description English,Job Description Extracted,Platform,Salary_E,Requirements,Nice to have,Responsibilities,Benefits
0,Austria,Europe,True,True,local,google.at,Sandbox Innovation Sdn. Bhd. | Senior Mobile A...,Tideri Jobbörse,Austria,"Trabajo.org - Stellenangebote, Arbeit",...,iOS developer,en,We are a dynamic FinTech company headquartered...,"1. Platform: Android, iOS, Cross-platform \n2...","Android, iOS, Cross-platform",Not mentioned,- Educational Background: Bachelor's degree in...,- Flutter experience is a plus. \n - Progra...,"- Develop, test, and deploy high quality mobil...",- Young & dynamic workplace & culture (with of...
1,Austria,Europe,True,True,default,google.com,Mobile Application Developer,Pearson Carter,Austria,"Trabajo.org - Stellenangebote, Arbeit",...,Android developer,en,Lead Mobile Developer | Hyper Growth Startup |...,1. Platform: Cross-platform \n2. Salary: $160...,Cross-platform,"$160,000 + Super",Not mentioned,Not mentioned,Not mentioned,Not mentioned
2,Austria,Europe,True,True,default,google.com,"Android Developer – Kotlin (Austria based, Hyb...",Bitcoin Devs Company,"Vienna, Austria",Jobs3,...,Android developer,en,Overview:\r\nThe Android Developer – Kotlin po...,1. Platform: Android \n2. Salary: Not mention...,Android,Not mentioned,• Passionate about mobile platforms and transl...,Not mentioned,• Ensure that the app meets our quality standa...,Not mentioned
3,Austria,Europe,True,True,default,google.com,Android & iOS Developer,ventopay gmbh,Austria,StudySmarter - Talents,...,iOS developer,de,**What are your tasks?** \n- You design attra...,1. Platform: Android/iOS \n2. Salary: Not men...,Android/iOS,Not mentioned,- A completed IT education (HTL/FH/University)...,- Professional experience in developing native...,- You design attractive modules and software p...,"- We are a stable, owner-managed company with ..."
4,Austria,Europe,True,True,default,google.com,iOS Developer Up3 (f/m/d),Drei Österreich,"Vienna, Austria","MyAbility.jobs, Drei., Jobted.at",...,iOS developer,en,Do you want to push the frontier of digital se...,1. Platform: iOS \n2. Salary: The gross annua...,iOS,The gross annual salary according to the colle...,• At least 3 years of experience as iOS develo...,Not mentioned,• Develop the iOS app of our fully digital pro...,• Top mobile phone of your choice incl. employ...


## Keep only iOS and Android vacancies 

In [28]:
platform_counts = df_sections["Platform"].value_counts().reset_index()
platform_counts.columns = ['Platform', 'Count']
print(platform_counts)

                        Platform  Count
0                            iOS   1094
1                        Android   1080
2                   Android, iOS    156
3   Android, iOS, Cross-platform    109
4                  Not mentioned     77
..                           ...    ...
67   iOS, Android, Windows Phone      1
68        Android/Cross-platform      1
69       Android, iOS, Web-based      1
70               Android, Huawei      1
71           Android, macOS, iOS      1

[72 rows x 2 columns]


In [29]:
df_sections[(df_sections['Platform'] == 'iOS/macOS') | (df_sections['Platform'] == 'iOS, tvOS')]

,Location,Region,EU Member,Schengen Agreement,Google Domain Type,Google Domain Used,Job Title,Company Name,Job Location,Apply Options,...,Search Query,Language,Job Description English,Job Description Extracted,Platform,Salary_E,Requirements,Nice to have,Responsibilities,Benefits
324,Canada,Northern America,False,False,default,google.com,iOS Developer (Remote),McAfee,"Waterloo, ON, Canada",Blind,...,iOS developer,en,Role OverviewMcAfee is searching for an interm...,1. Platform: iOS/macOS \n2. Salary: Not menti...,iOS/macOS,Not mentioned,- You can develop iOS applications at an inter...,- Diving deep into lower-level libraries like ...,- Dive deep into anti-censorship technologies ...,Not mentioned
385,Canada,Northern America,False,False,default,google.com,"Software Developer in Test, Creativity Apps",Apple,"Vancouver, BC, Canada",Careers At Apple,...,iOS developer,en,"Summary\r\nPosted: Sep 4, 2024\r\n\r\nRole Num...","1. Platform: iOS/macOS \n2. Salary: $113,400 ...",iOS/macOS,"$113,400 and $215,300",• 5+ years experience in QA/QE \n • Minimum...,• 2+ years of Full-stack Software Developer in...,"In this role, you will be responsible for plan...","• At Apple, base pay is one part of our total ..."
534,Czechia,Europe,True,True,default,google.com,iOS Developer,Sledovanitv.cz,"Brno, Czechia",Indeed.cz,...,iOS developer,cs,We are SledovaniTV.cz - the most technological...,"1. Platform: iOS, tvOS \n2. Salary: Not menti...","iOS, tvOS",Not mentioned,- Advanced knowledge of iOS development (exper...,- Your own project to showcase,- Have the opportunity to dive into our iOS an...,"- The opportunity to work in a young, inspirin..."
1273,Mexico,Northern America,False,False,default,google.com,Senior Ios Developer,Bhuvi It Solutions,"Guadalajara, Jalisco, Mexico","BeBee, Trabajo.org - Vacantes De Empleo, Traba...",...,iOS developer,en,Job Title: iOS Developer\r\n\r\nWe are seeking...,"1. Platform: iOS, tvOS \n2. Salary: $120,000 ...","iOS, tvOS","$120,000 - $180,000 per year.",• 4+ years of professional software developmen...,Not mentioned,"• Produce a reliable, performant, configurable...",• TN Visa Sponsorship. \n • USDPay. \n •...


### Rename rows with 'iOS/macOS' or 'iOS, tvOS' to iOS

In [ ]:
# Replace the first two rows with 'iOS, tvOS'
df_sections.loc[df_sections[df_sections['Platform'] == 'iOS, tvOS'].iloc[:2].index, 'Platform'] = 'iOS'
# Replace the first two rows with 'iOS/macOS'
df_sections.loc[df_sections[df_sections['Platform'] == 'iOS/macOS'].iloc[:2].index, 'Platform'] = 'iOS'

### Keep only iOS and Android vacancies

In [31]:
df_filtered = df_sections.copy()

df_filtered = df_filtered[df_filtered['Platform'].isin(['iOS', 'Android'])]
df_filtered = df_filtered.sort_values(by='Location').reset_index(drop=True)
df_filtered.head()

,Location,Region,EU Member,Schengen Agreement,Google Domain Type,Google Domain Used,Job Title,Company Name,Job Location,Apply Options,...,Search Query,Language,Job Description English,Job Description Extracted,Platform,Salary_E,Requirements,Nice to have,Responsibilities,Benefits
0,Austria,Europe,True,True,default,google.com,"Android Developer – Kotlin (Austria based, Hyb...",Bitcoin Devs Company,"Vienna, Austria",Jobs3,...,Android developer,en,Overview:\r\nThe Android Developer – Kotlin po...,1. Platform: Android \n2. Salary: Not mention...,Android,Not mentioned,• Passionate about mobile platforms and transl...,Not mentioned,• Ensure that the app meets our quality standa...,Not mentioned
1,Austria,Europe,True,True,default,google.com,ios entwickler 80–100% w/m/d,CHANCENLAND VORARLBERG,"Dornbirn, Austria","IT-Career.at, STEMJOBS.AT, IT-JOBS.AT",...,iOS developer,de,**iOS Developer 80–100% w/m/d**\n\n**Job Descr...,1. Platform: iOS \n2. Salary: Not mentioned ...,iOS,Not mentioned,"- **Motivation over experience:** Curiosity, i...",Not mentioned,- You will work with us on exciting projects f...,"- A compact, battle-tested team and flat hiera..."
2,Austria,Europe,True,True,default,google.com,Middle iOS developer,Processica,"Vienna, Austria",JOBITT,...,iOS developer,en,Looking for a iOS Developer. Playing well in a...,1. Platform: iOS \n2. Salary: Not mentioned ...,iOS,Not mentioned,Playing well in a team and has strong analytic...,Not mentioned,Not mentioned,Not mentioned
3,Austria,Europe,True,True,default,google.com,iOS Developer,Raiffeisen Gruppe,"Linz, Austria","Jooble, Trabajo.org - Stellenangebote, Arbeit",...,iOS developer,de,**Your Role in the Team** \n- You contribute ...,1. Platform: iOS \n2. Salary: Not mentioned ...,iOS,Not mentioned,- You are familiar with Continuous Integration...,Not mentioned,- You contribute to the further development of...,- Home Office \n - Flexible working hours ...
4,Austria,Europe,True,True,default,google.com,iOS Software Engineer,Cybermoth,"Vienna, Austria","Expertini, Talent.com",...,iOS developer,en,We are searching for iOS Software Engineers wi...,1. Platform: iOS \n2. Salary: Depending on qu...,iOS,Depending on qualifications and professional e...,• Worked on at least one native Swift applicat...,Not mentioned,• In this position you will be part of one or ...,• You will be part of a company with an inspir...


## Extract technologies and tools

### Ground Truth testing

In [ ]:
df_test = df_filtered.copy()

df_extract_sample = df_test.sample(n=40, random_state=42)

In [223]:
df_extract_sample["Full Requirements"] = (
    "3. Requirements:\n" + df_extract_sample["Requirements"].astype(str) + "\n\n" +
    "4. Nice to have:\n" + df_extract_sample["Nice to have"].astype(str) + "\n\n" +
    "5. Responsibilities:\n" + df_extract_sample["Responsibilities"].astype(str)
)
df_extract_sample

,Location,Region,EU Member,Schengen Agreement,Google Domain Type,Google Domain Used,Job Title,Company Name,Job Location,Apply Options,...,Language Name,Job Description English,Job Description Extracted,Platform,Salary_E,Requirements,Nice to have,Responsibilities,Benefits,Full Requirements
282,Canada,Northern America,False,False,default,google.com,Développeur iOS senior -Senior iOS Developer,OneSpan,"Montreal, Quebec, Canada","Greenhouse, Indeed, Kovasys IT Recruitment Age...",...,French,The English translation for the job descriptio...,1. Platform: iOS \n2. Salary: Not mentioned ...,iOS,Not mentioned,Not mentioned,Not mentioned,Not mentioned,Not mentioned,3. Requirements:\nNot mentioned\n\n4. Nice to ...
479,France,Europe,True,True,default,google.com,Senior Android Developer - Video Expert (All G...,Dailymotion,"Paris, France",Ivy Exec,...,English,Company Description\r\n\r\nDailymotion is the ...,1. Platform: Android \n2. Salary: Not mention...,Android,Not mentioned,• Excellent knowledge of Android SDK \n • A...,• Proven experience in developing video-based ...,• Design and build advanced applications for t...,Not mentioned,3. Requirements:\n• Excellent knowledge of And...
1307,Romania,Europe,True,True,local,google.ro,iOS Developer,ASSIST SOFTWARE,"Suceava, Romania",IOS Jobs,...,English,Are you an iOS mobile innovator? We need you! ...,1. Platform: iOS \n2. Salary: Competitive sal...,iOS,Competitive salary,• Ability to design and implement iOS applicat...,"• Experience with Phonegap, HTML5 \n • Adep...",Not mentioned,Relocation package for 2 months \n Employme...,3. Requirements:\n• Ability to design and impl...
1254,Portugal,Europe,True,True,default,google.com,Principal iOS Software Engineer,Sky,"Aveiro, Portugal",Empregos Trabajo.org,...,English,Role Overview\r\n\r\nSky Portugal is a leading...,"1. Platform: iOS \n2. Salary: $120,000 - $180...",iOS,"$120,000 - $180,000 per annum",• 5+ years of experience in native iOS develop...,Not mentioned,"• Design, develop, and deliver high-quality so...",• Above market salary \n • Yearly bonus \n...,3. Requirements:\n• 5+ years of experience in ...
507,Germany,Europe,True,True,default,google.com,iOS Developer - Health Care App (m/w/d) Trier,HIBA GmbH,"Trier, Germany",XING,...,German,**iOS Developer - Health Care App (m/f/d) Trie...,1. Platform: iOS \n2. Salary: Not mentioned ...,iOS,Not mentioned,- Completed degree in computer science or a co...,Not mentioned,- Development and maintenance of iOS applicati...,- Continuous training opportunities to ensure ...,3. Requirements:\n- Completed degree in comput...
495,France,Europe,True,True,default,google.com,Android (Kotlin) Developer - Freelance,Veepee,"Nantes, France",Okko Jobs,...,English,The vente-privee group has consolidated its va...,1. Platform: Android \n2. Salary: Not mention...,Android,Not mentioned,• At least 3 years of experience in building A...,Not mentioned,• Propose and work on new solutions with respe...,• Large amount of good quality data (over 1M v...,3. Requirements:\n• At least 3 years of experi...
416,Estonia,Europe,True,True,default,google.com,iOS Software Development Expert,Addendum,"Tallinn, Estonia","BeBee, Trabajo.org",...,English,About the Company\r\n\r\nAddendum is a global ...,1. Platform: iOS \n2. Salary: $40-$80 per hou...,iOS,$40-$80 per hour,• Proven experience in developing native iOS a...,Not mentioned,• Develop high-quality native iOS applications...,• Flexible working hours. \n • Top talent c...,3. Requirements:\n• Proven experience in devel...
2009,United States,Northern America,False,False,default,google.com,iOS Developer,CloudBerg Tec,"Salt Lake City, UT","Indeed, Glassdoor, Jooble, OPTnation, LinkedIn...",...,English,Job Overviews\r\n\r\nLocation:\r\nSalt Lake Ci...,1. Platform: iOS \n2. Salary: Not mentioned ...,iOS,Not mentioned,- We are looking for a passionate iOS develope...,Not mentioned,- Designing and building mobile applications f...,Not mentioned,3. Requirements:\n- We are looking for a pa

#### Dataframe into Text

In [224]:
#column_to_txt(column='Full Requirements', output="./ground truth/Full_requirements_sample.txt", df=df_extract_sample)

####  Load Ground Truth into a DataFrame

In [225]:
df_extract_sample = txt_to_column(txt_path="./ground truth/Full_requirements_ground_truth.txt", df=df_extract_sample)
df_extract_sample.head()

,Location,Region,EU Member,Schengen Agreement,Google Domain Type,Google Domain Used,Job Title,Company Name,Job Location,Apply Options,...,Job Description English,Job Description Extracted,Platform,Salary_E,Requirements,Nice to have,Responsibilities,Benefits,Full Requirements,Ground Truth
282,Canada,Northern America,False,False,default,google.com,Développeur iOS senior -Senior iOS Developer,OneSpan,"Montreal, Quebec, Canada","Greenhouse, Indeed, Kovasys IT Recruitment Age...",...,The English translation for the job descriptio...,1. Platform: iOS \n2. Salary: Not mentioned ...,iOS,Not mentioned,Not mentioned,Not mentioned,Not mentioned,Not mentioned,3. Requirements:\nNot mentioned\n\n4. Nice to ...,{}
479,France,Europe,True,True,default,google.com,Senior Android Developer - Video Expert (All G...,Dailymotion,"Paris, France",Ivy Exec,...,Company Description\r\n\r\nDailymotion is the ...,1. Platform: Android \n2. Salary: Not mention...,Android,Not mentioned,• Excellent knowledge of Android SDK \n • A...,• Proven experience in developing video-based ...,• Design and build advanced applications for t...,Not mentioned,3. Requirements:\n• Excellent knowledge of And...,"{\n ""languages_and_runtimes"": [""Kotlin"", ""co..."
1307,Romania,Europe,True,True,local,google.ro,iOS Developer,ASSIST SOFTWARE,"Suceava, Romania",IOS Jobs,...,Are you an iOS mobile innovator? We need you! ...,1. Platform: iOS \n2. Salary: Competitive sal...,iOS,Competitive salary,• Ability to design and implement iOS applicat...,"• Experience with Phonegap, HTML5 \n • Adep...",Not mentioned,Relocation package for 2 months \n Employme...,3. Requirements:\n• Ability to design and impl...,"{\n ""languages_and_runtimes"": [""Objective-C""..."
1254,Portugal,Europe,True,True,default,google.com,Principal iOS Software Engineer,Sky,"Aveiro, Portugal",Empregos Trabajo.org,...,Role Overview\r\n\r\nSky Portugal is a leading...,"1. Platform: iOS \n2. Salary: $120,000 - $180...",iOS,"$120,000 - $180,000 per annum",• 5+ years of experience in native iOS develop...,Not mentioned,"• Design, develop, and deliver high-quality so...",• Above market salary \n • Yearly bonus \n...,3. Requirements:\n• 5+ years of experience in ...,"{\n ""languages_and_runtimes"":[""Swift""],\n ..."
507,Germany,Europe,True,True,default,google.com,iOS Developer - Health Care App (m/w/d) Trier,HIBA GmbH,"Trier, Germany",XING,...,**iOS Developer - Health Care App (m/f/d) Trie...,1. Platform: iOS \n2. Salary: Not mentioned ...,iOS,Not mentioned,- Completed degree in computer science or a co...,Not mentioned,- Development and maintenance of iOS applicati...,- Continuous training opportunities to ensure ...,3. Requirements:\n- Completed degree in comput...,"{\n ""languages_and_runtimes"": [""Objective-C""..."


#### Chat GPT's version 

In [226]:
system_prompt_1 = """
You are a senior extraction assistant.

GOAL
Given a natural-language job-vacancy text, return one and only one
valid, minified JSON object that maps every technology term found in
the input to exactly ONE key from ALLOWED_KEYS.

STRICT RULES
1. Output = raw JSON only (no Markdown, no comments, no extra text).
2. Use keys from ALLOWED_KEYS exactly as written. Omit any key whose array would be empty.
3. Each value is an **array of unique strings**, sorted alphabetically and preserving the term’s original spelling/case from the input.
4. Ignore soft skills and vague nouns (e.g. communication, documentation, performance, detail, English, collaboration, problem-solving, university).
5. **IGNORE** overly generic technology words/phrases (e.g. "design patterns", "android ui", "json", "xml", "clean code", "API integration", "unit testing", "Continuous integration", "CI", "CD", "CI/CD", "cryptography", "application testing" ect.).
6. DO NOT hallucinate. Include a term **only** if it appears verbatim in the input text.
7. If a technology term matches **exactly** (case-insensitive) one of the values in ALLOWED_KEYS, then it MUST be assigned to that exact key — even if the name could hypothetically fit other categories.
8. The order of keys in the output JSON MUST strictly follow the order of keys as they appear in the ALLOWED_KEYS section below. Only include keys that have at least one matched term. If a key has no matched terms, it MUST be completely omitted from the output, even if this breaks the visual continuity of the ALLOWED_KEYS order.


ALLOWED_KEYS (with examples)
{
    "languages_and_runtimes": ["Swift", "Kotlin", "Java", "Dart", "Objective-C", "Coroutines", "RxSwift", "Combine", "GCD", "RxJava", "Flows", "JavaScript", "TypeScript", "C", "C++", "Python", "Golang", "PHP"],
    "ui_and_cross_platform_frameworks": ["SwiftUI", "UIKit", "Jetpack Compose", "Flutter", "React Native", "Xamarin", "Ionic", "WatchKit", "Cocoa Touch", "PhoneGap", "Cordova", "Kotlin Multiplatform"],
    "architectural_patterns": ["MVVM", "VIPER", "Clean Architecture", "MVI", "MVC", "MVP", "Redux"],
    "dependency_injection_frameworks": ["Dagger", "Hilt", "Koin", "Swinject"],
    "build_and_dependency_management": ["Gradle", "CocoaPods", "SPM", "Bazel", "Buck", "Xcode", "Android Studio", "CircleCI", "Bamboo", "CocoaPods"],
    "data_and_caching": ["Core Data", "Room", "Realm", "SQLite", "Firestore", "MongoDB", "SAP UltraLite", "MySQL", "NoSQL"],
    "networking_and_api": ["Retrofit", "OkHttp", "URLSession", "Alamofire", "GraphQL", "REST API", "WebSockets", "gRPC", "Protocol Buffers"],
    "backend_or_baas_integration": ["Firebase", "AWS Amplify", "Azure Mobile", "Parse", "AWS Mobile Hub", "AWS Cognito", "AWS S3"],
    "device_and_platform_services_and_third_party_sdks": ["ARKit", "HealthKit", "CoreML", "Core Animation", "Android SDK", "Android NDK", "Push Notifications", "BLE", "NFC", "Camera", "Location", "Sensors", "Stripe SDK", "Facebook SDK", "AdMob", "Google Maps", "ExoPlayer", "Glide", "Stripe", "PayPal SDK", "Binder", "AIDL", "JNI", "CTS", "HAL", "LoRa", "CarPlay", "Android Auto", "CydiaSubstrate", "Frida", "WebViews"],
    "security_and_cryptography": ["Keychain", "TLS pinning", "OAuth2", "Veracode", "Checkmarx", "OWASP"],
    "testing_frameworks": ["XCTest", "JUnit", "Espresso", "Mockito", "Robolectric"],
    "debugging_or_profiling": ["Instruments", "Android Profiler"],
    "version_control": ["Git", "GIT", "git", "SVN", "Mercurial", "Gerrit", "GitFlow", "SourceTree", "Fork", "Bitbucket"],
    "ci_cd_and_release_automation": ["GitHub Actions", "Jenkins", "Bitrise", "fastlane", "CircleCI", "Bamboo", "GitLab CI", "Docker"],
    "monitoring_analytics_and_crash_reporting": ["Crashlytics", "Sentry", "Datadog", "Firebase Analytics", "App Center"],
    "development_methodologies": ["Scrum", "Kanban", "Agile", "SAFe", "TDD", "BDD", "DevOps"],
    "testing_process_and_qa": ["test coverage", "regression testing"],
    "code_quality_and_static_analysis": ["SonarQube", "SwiftLint", "Veracode", "Checkmarx"],
    "documentation_and_knowledge_sharing": ["Swagger", "OpenAPI", "Javadoc", "Confluence", "HIG"],
    "collaboration_pm_and_design_handoff": ["Jira", "Trello", "Figma", "Zeplin", "Rally/AgileCentral"],
    "distribution_and_store_operations": ["TestFlight", "App Store Connect", "Google Play Console"],
    "compliance_and_certifications": ["SOC 2", "GDPR"]
}

EXAMPLE RESPONSE
{
    "languages_and_runtimes":["Swift","Java"],
    "version_control":["Git"]
}
"""


user_prompt_1 = """
Extract every technology, tool, framework, library, service or formal
methodology mentioned in the text below and output the JSON exactly as
specified in the system prompt.

---
INPUT TEXT:
"""


df_extracted_requirements_test = await chatgpt_async(
    input_column_name="Full Requirements", 
    output_column_name="Extracted Technologies GPT",
    input_text_length=None,
    output_text_length=None,
    num_rows=None,  
    df=df_extract_sample.copy(), 
    system_prompt=system_prompt_1,
    user_prompt=user_prompt_1,
    gpt_model="gpt-4o-2024-11-20",
    client=client_async,
    batch_size=40,
    concurrency_limit=40,
    cache_file= "./cache/Full_requirements_cache.json"
)

Processing Batches: 100%|██████████| 1/1 [00:00<00:00, 23.32it/s]


In [227]:
df_extracted_requirements_test.head()

,Location,Region,EU Member,Schengen Agreement,Google Domain Type,Google Domain Used,Job Title,Company Name,Job Location,Apply Options,...,Job Description Extracted,Platform,Salary_E,Requirements,Nice to have,Responsibilities,Benefits,Full Requirements,Ground Truth,Extracted Technologies GPT
282,Canada,Northern America,False,False,default,google.com,Développeur iOS senior -Senior iOS Developer,OneSpan,"Montreal, Quebec, Canada","Greenhouse, Indeed, Kovasys IT Recruitment Age...",...,1. Platform: iOS \n2. Salary: Not mentioned ...,iOS,Not mentioned,Not mentioned,Not mentioned,Not mentioned,Not mentioned,3. Requirements:\nNot mentioned\n\n4. Nice to ...,{},{}
479,France,Europe,True,True,default,google.com,Senior Android Developer - Video Expert (All G...,Dailymotion,"Paris, France",Ivy Exec,...,1. Platform: Android \n2. Salary: Not mention...,Android,Not mentioned,• Excellent knowledge of Android SDK \n • A...,• Proven experience in developing video-based ...,• Design and build advanced applications for t...,Not mentioned,3. Requirements:\n• Excellent knowledge of And...,"{\n ""languages_and_runtimes"": [""Kotlin"", ""co...","{\n ""languages_and_runtimes"":[""Kotlin"",""Cor..."
1307,Romania,Europe,True,True,local,google.ro,iOS Developer,ASSIST SOFTWARE,"Suceava, Romania",IOS Jobs,...,1. Platform: iOS \n2. Salary: Competitive sal...,iOS,Competitive salary,• Ability to design and implement iOS applicat...,"• Experience with Phonegap, HTML5 \n • Adep...",Not mentioned,Relocation package for 2 months \n Employme...,3. Requirements:\n• Ability to design and impl...,"{\n ""languages_and_runtimes"": [""Objective-C""...","{\n ""languages_and_runtimes"":[""Objective-C""..."
1254,Portugal,Europe,True,True,default,google.com,Principal iOS Software Engineer,Sky,"Aveiro, Portugal",Empregos Trabajo.org,...,"1. Platform: iOS \n2. Salary: $120,000 - $180...",iOS,"$120,000 - $180,000 per annum",• 5+ years of experience in native iOS develop...,Not mentioned,"• Design, develop, and deliver high-quality so...",• Above market salary \n • Yearly bonus \n...,3. Requirements:\n• 5+ years of experience in ...,"{\n ""languages_and_runtimes"":[""Swift""],\n ...","{\n ""languages_and_runtimes"":[""Swift""],\n ..."
507,Germany,Europe,True,True,default,google.com,iOS Developer - Health Care App (m/w/d) Trier,HIBA GmbH,"Trier, Germany",XING,...,1. Platform: iOS \n2. Salary: Not mentioned ...,iOS,Not mentioned,- Completed degree in computer science or a co...,Not mentioned,- Development and maintenance of iOS applicati...,- Continuous training opportunities to ensure ...,3. Requirements:\n- Completed degree in comput...,"{\n ""languages_and_runtimes"": [""Objective-C""...","{\n ""languages_and_runtimes"":[""Objective-C""..."


### Token set ratio

In [228]:
def extract_values(cell):
    """
    Превращает JSON-ячейку в плоский список строковых значений.
    Если ячейка пустая / некорректная — возвращает [].
    """
    if pd.isna(cell) or str(cell).strip() == "":
        return []

    try:
        parsed = json.loads(cell)
    except (ValueError, TypeError):
        return []

    values = []
    if isinstance(parsed, dict):
        for v in parsed.values():
            if isinstance(v, list):
                values.extend(v)
            elif v is not None:
                values.append(v)
    elif isinstance(parsed, list):
        values.extend(parsed)
    else:
        values.append(parsed)

    # нормализуем так же, как делает RapidFuzz внутри token_set_ratio
    return [utils.default_process(str(x)) for x in values if str(x).strip()]

def technologies_token_set_ratio(gt_cell, gpt_cell):
    """
    Считает token_set_ratio для двух JSON-ячееĸ,
    используя только значения.
    """
    vals_gt  = extract_values(gt_cell)
    vals_gpt = extract_values(gpt_cell)

    # оба пусты → полное совпадение
    if not vals_gt and not vals_gpt:
        return 100

    s_gt  = " ".join(vals_gt)
    s_gpt = " ".join(vals_gpt)
    return fuzz.token_set_ratio(s_gt, s_gpt)


df_extracted_requirements_test["Token Set Ratio"] = (df_extracted_requirements_test.apply(lambda row: technologies_token_set_ratio(row["Ground Truth"], row["Extracted Technologies GPT"]),axis=1,).round(2))
print("Average Token Set Ratio :", df_extracted_requirements_test["Token Set Ratio"].mean().round(2))


Average Token Set Ratio : 96.73


### Full extraction

In [229]:
df_final_extraction = df_final.copy()

In [230]:
df_final_extraction["Full Requirements"] = (
    "3. Requirements:\n" + df_final_extraction["Requirements"].astype(str) + "\n\n" +
    "4. Nice to have:\n" + df_final_extraction["Nice to have"].astype(str) + "\n\n" +
    "5. Responsibilities:\n" + df_final_extraction["Responsibilities"].astype(str)
)

In [231]:
df_final_extracted = await chatgpt_async(
    input_column_name="Full Requirements", 
    output_column_name="Extracted Technologies GPT",
    input_text_length=None,
    output_text_length=None,
    num_rows=None,  
    df=df_final_extraction.copy(), 
    system_prompt=system_prompt_1,
    user_prompt=user_prompt_1,
    gpt_model="gpt-4o-2024-11-20",
    client=client_async,
    batch_size=25,
    concurrency_limit=25,
    cache_file= "./cache/Full_requirements_cache.json"
)

Processing Batches: 100%|██████████| 88/88 [00:03<00:00, 28.75it/s]


In [232]:
df_final_extracted.head()

,Location,Region,EU Member,Schengen Agreement,Google Domain Type,Google Domain Used,Job Title,Company Name,Job Location,Apply Options,...,Job Description English,Job Description Extracted,Platform,Salary_E,Requirements,Nice to have,Responsibilities,Benefits,Full Requirements,Extracted Technologies GPT
0,Austria,Europe,True,True,default,google.com,"Android Developer – Kotlin (Austria based, Hyb...",Bitcoin Devs Company,"Vienna, Austria",Jobs3,...,Overview:\r\nThe Android Developer – Kotlin po...,1. Platform: Android \n2. Salary: Not mention...,Android,Not mentioned,• Passionate about mobile platforms and transl...,Not mentioned,• Ensure that the app meets our quality standa...,Not mentioned,3. Requirements:\n• Passionate about mobile pl...,"{\n ""languages_and_runtimes"":[""Kotlin"",""Cor..."
1,Austria,Europe,True,True,default,google.com,ios entwickler 80–100% w/m/d,CHANCENLAND VORARLBERG,"Dornbirn, Austria","IT-Career.at, STEMJOBS.AT, IT-JOBS.AT",...,**iOS Developer 80–100% w/m/d**\n\n**Job Descr...,1. Platform: iOS \n2. Salary: Not mentioned ...,iOS,Not mentioned,"- **Motivation over experience:** Curiosity, i...",Not mentioned,- You will work with us on exciting projects f...,"- A compact, battle-tested team and flat hiera...",3. Requirements:\n- **Motivation over experien...,{}
2,Austria,Europe,True,True,default,google.com,Middle iOS developer,Processica,"Vienna, Austria",JOBITT,...,Looking for a iOS Developer. Playing well in a...,1. Platform: iOS \n2. Salary: Not mentioned ...,iOS,Not mentioned,Playing well in a team and has strong analytic...,Not mentioned,Not mentioned,Not mentioned,3. Requirements:\nPlaying well in a team and h...,{}
3,Austria,Europe,True,True,default,google.com,iOS Developer,Raiffeisen Gruppe,"Linz, Austria","Jooble, Trabajo.org - Stellenangebote, Arbeit",...,**Your Role in the Team** \n- You contribute ...,1. Platform: iOS \n2. Salary: Not mentioned ...,iOS,Not mentioned,- You are familiar with Continuous Integration...,Not mentioned,- You contribute to the further development of...,- Home Office \n - Flexible working hours ...,3. Requirements:\n- You are familiar with Cont...,"{\n ""languages_and_runtimes"":[""Swift""],\n ..."
4,Austria,Europe,True,True,default,google.com,iOS Software Engineer,Cybermoth,"Vienna, Austria","Expertini, Talent.com",...,We are searching for iOS Software Engineers wi...,1. Platform: iOS \n2. Salary: Depending on qu...,iOS,Depending on qualifications and professional e...,• Worked on at least one native Swift applicat...,Not mentioned,• In this position you will be part of one or ...,• You will be part of a company with an inspir...,3. Requirements:\n• Worked on at least one nat...,"{\n ""languages_and_runtimes"":[""Java"",""Kotli..."


---

## Remove hallucinated words

In [233]:
# ── 0. Настройки ────────────────────────────────────────────────
STANDARDIZE = True            # ⬅ выключи, если нужно оставить исходные формы
USE_REGEX   = False           # ⬅ как и раньше

# ── 1. Загружаем словарь и строим «вариант → канон» ─────────────
with open(".//synonyms.json", encoding="utf8") as f:
    SYNONYMS = json.load(f)

VAR2CANON = {v.lower(): canon      # "m-v-c"  -> "mvc"
             for canon, variants in SYNONYMS.items()
             for v in variants}

# ── 2. Вспомогалки ──────────────────────────────────────────────
def make_ngrams(tokens, n_max=3):
    return {" ".join(tokens[i:i+n])
            for n in range(1, n_max + 1)
            for i in range(len(tokens) - n + 1)}

def is_real(term, text_flat, ngrams,
            *, use_regex=USE_REGEX, synonyms=SYNONYMS):
    term_l = term.lower()
    if term_l not in synonyms:           # нет в списке «проверяемых» → сразу True
        return True

    patterns = synonyms[term_l]
    if any(p.lower() in ngrams for p in patterns):
        return True
    if use_regex and any(re.search(rf"\b{p}\b", text_flat) for p in patterns):
        return True
    return False

# ── 3. Счётчики ────────────────────────────────────────────────
removed_counter       = Counter()   # что удалили как галлюцинацию
canonicalized_counter = Counter()   # сколько раз заменили на канон

def to_canon(term):
    """Если STANDARDIZE=True и вариант есть в словаре → вернём канон."""
    t_low = term.lower()
    if STANDARDIZE and t_low in VAR2CANON:
        canon = VAR2CANON[t_low]
        if canon != t_low:          # реальная замена, а не «mvc» -> «mvc»
            canonicalized_counter[canon] += 1
        return canon
    return t_low                    # либо исходную форму (в lower)

# ── 4. Фильтр ──────────────────────────────────────────────────
def remove_hallucinated(row):
    text_raw   = (row.get("Full Requirements") or "")
    text_lower = text_raw.lower()

    token_pattern = r"\.?[a-z0-9\+\#-]+(?:\.[a-z0-9\+\#-]+)*"
    tokens = re.findall(token_pattern, text_lower)

    ngrams  = make_ngrams(tokens)

    extracted = row.get("Extracted Technologies GPT", "")
    try:
        tech_dict = json.loads(extracted)
    except Exception:
        return extracted            # повреждённый JSON

    for cat in list(tech_dict.keys()):
        cleaned = []
        for term in tech_dict[cat]:
            canon_term = to_canon(term)
            if is_real(canon_term, text_lower, ngrams):
                cleaned.append(canon_term)
            else:
                removed_counter[canon_term] += 1

        if cleaned:
            tech_dict[cat] = list(dict.fromkeys(cleaned))  # убираем дубли
        else:
            del tech_dict[cat]

    return json.dumps(tech_dict, ensure_ascii=False)

# ── 5. Запуск ──────────────────────────────────────────────────
df_final_extracted["Extracted Technologies Clean"] = (
    df_final_extracted.apply(remove_hallucinated, axis=1)
)

# ── 6. Итоговая статистика ─────────────────────────────────────
print("\n== Hallucinations removed ==")
print(pd.DataFrame(removed_counter.items(),
                   columns=["hallucinated_term", "removed_count"])
        .sort_values("removed_count", ascending=False)
        .reset_index(drop=True))

print("\n== The terms are canonized ==")
print(pd.DataFrame(canonicalized_counter.items(),
                   columns=["canonical_term", "replaced_count"])
        .sort_values("replaced_count", ascending=False)
        .reset_index(drop=True))

print(f"\nTotal rows processed: {len(df_final_extracted)}")
print(f"Total hallucinated terms removed: {sum(removed_counter.values())}")
print(f"Total terms canonicalized: {sum(canonicalized_counter.values())}")


== Hallucinations removed ==
                          hallucinated_term  removed_count
0                                       git             64
1                               android sdk             55
2                                       hig             37
3                                     swift             27
4                            android studio             17
5                                     agile             17
6                                      mvvm             16
7                                    binder             13
8                                      java             11
9                                    dagger              9
10                       clean architecture              7
11                         android profiler              7
12                                   oauth2              7
13                              google maps              6
14                                      tdd              6
15                        

### Inspection

In [234]:
def flat_terms(tech_json: str) -> set[str]:
    """{"cat": ["A", "B"]}  → {"a", "b"}   (lower-case)"""
    try:
        d = json.loads(tech_json)
    except Exception:
        return set()
    return {t.lower() for lst in d.values() for t in lst}

# исходный → множество   |   очищенный → множество
orig_sets  = df_final_extracted["Extracted Technologies GPT"].apply(flat_terms)
clean_sets = df_final_extracted["Extracted Technologies Clean"].apply(flat_terms)

df_final_extracted["removed_terms"] = orig_sets.subtract(clean_sets)



term = "kotlin multiplatform"                       # или "google play console", "android sdk", …

mask = df_final_extracted["removed_terms"].apply(lambda s: term.lower() in s)

view_cols = ["Full Requirements",
             "Extracted Technologies GPT",
             "Extracted Technologies Clean"]

df_debug = df_final_extracted.loc[mask, view_cols]

print(f"Rows with removed '{term}': {len(df_debug)}")
df_debug.head(5)                 # покажет первые 3 совпадения


Rows with removed 'kotlin multiplatform': 3


,Full Requirements,Extracted Technologies GPT,Extracted Technologies Clean
9,3. Requirements:\n- 5+ years of industry exper...,"{\n ""languages_and_runtimes"":[""Objective-C""...","{""languages_and_runtimes"": [""objective-c"", ""sw..."
765,3. Requirements:\n• Bachelor's degree in Compu...,"{\n ""languages_and_runtimes"":[""C"",""Coroutin...","{""languages_and_runtimes"": [""c"", ""kotlin corou..."
943,3. Requirements:\n- Bachelor’s degree in Compu...,"{\n ""languages_and_runtimes"":[""Java"",""Kotli...","{""languages_and_runtimes"": [""java"", ""kotlin""],..."


In [235]:
df_final_extracted = df_final_extracted.drop(columns=['removed_terms'])

In [236]:
#pd.set_option('display.max_rows', 200)

## Cleaning and Normolization

### Extract values

In [237]:
# Функция для извлечения значений из JSON-строки
def extract_values(json_str):
    if pd.isna(json_str) or json_str.strip() == '{}' or json_str.strip() == '':
        return None
    try:
        data = json.loads(json_str)
        values = []
        for val in data.values():
            if isinstance(val, list):
                values.extend(map(str, val))  # добавляем элементы списка как строки
            else:
                values.append(str(val))  # добавляем одиночное значение как строку
        return ', '.join(values) if values else None
    except json.JSONDecodeError:
        return None

In [238]:
df_final_extracted['Technologies Only'] = df_final_extracted['Extracted Technologies Clean'].apply(extract_values).str.lower()

### Cleaning and Normolization

In [239]:
with open(".//remove_list.json", encoding="utf8") as f:
    remove_list = json.load(f)

# Функция для нормализации строки
def normalize_tech_string(tech_str):
    if pd.isna(tech_str):
        return None
    try:
        tech_list = tech_str.split(',')
        clean_terms = []
        for term in tech_list:
            term_clean = term.strip().lower()
            if term_clean not in remove_list:
                clean_terms.append(term_clean)
        return ', '.join(clean_terms) if clean_terms else None
    except Exception:
        return None

# Применяем к колонке
df_final_extracted['Technologies Only'] = df_final_extracted['Technologies Only'].apply(normalize_tech_string)

In [240]:
# Удалим пропуски, если есть
tech_series = df_final_extracted['Technologies Only'].dropna()

# Разделим строки по запятой, удалим лишние пробелы, соберём всё в одну серию
all_techs = tech_series.str.split(',').explode().str.strip()

# Посчитаем частоты
tech_counts = all_techs.value_counts().reset_index()



tech_counts = tech_counts[tech_counts["count"] >= 2]

tech_counts

,Technologies Only,count
0,swift,914
1,kotlin,853
2,git,644
3,java,600
4,rest api,518
...,...,...
209,yocto,2
210,spring boot,2
211,npm,2
212,tls pinning,2


### Cheking

In [241]:
# Загрузка данных
with open('.\\key_values.json', 'r') as f:
    json_data = json.load(f)

# Получение списков технологий
tech_list = tech_counts['Technologies Only'].str.strip().str.lower().unique().tolist()
json_tech_list = []
for category, items in json_data.items():
    json_tech_list.extend([item.strip().lower() for item in items])

tech_set = set(tech_list)
json_set = set(json_tech_list)

# Поиск различий
only_in_tech_counts = tech_set - json_set
only_in_json = json_set - tech_set

# Вывод сводки
print(f"\n\nСводка сравнения:")
print(f"Всего технологий в tech_counts: {len(tech_set)}")
print(f"Всего технологий в JSON: {len(json_set)}")
print(f"Технологий только в tech_counts: {len(only_in_tech_counts)}")
print(f"Технологий только в JSON: {len(only_in_json)}")

if only_in_tech_counts:
    print("\nТехнологии в tech_counts, но отсутствуют в JSON:")
    for tech in sorted(only_in_tech_counts):
        print(f"- {tech}")

if only_in_json:
    print("\nТехнологии в JSON, но отсутствуют в tech_counts:")
    for tech in sorted(only_in_json):
        print(f"- {tech}")




Сводка сравнения:
Всего технологий в tech_counts: 214
Всего технологий в JSON: 214
Технологий только в tech_counts: 0
Технологий только в JSON: 0


### Deploy

In [242]:
with open('.\\key_values.json', 'r', encoding='utf-8') as f:
    kv = json.load(f)                  # categories → [tech1, tech2, …]

# -----------------------------------------------------------
# 2. Готовим «обратный» словарь: tech (lower) → category name
# -----------------------------------------------------------
reverse_map = {
    tech.lower(): category
    for category, tech_list in kv.items()
    for tech in tech_list
}

# -----------------------------------------------------------
# 3. Функция, которая строит JSON-объект для одной ячейки
# -----------------------------------------------------------
def categorize(tech_cell: str) -> dict:
    if pd.isna(tech_cell) or not tech_cell.strip():
        return {}
    
    result = {}
    # «kotlin, retrofit » → ['kotlin', 'retrofit']
    for raw in tech_cell.split(','):
        tech = raw.strip()
        if not tech:
            continue
        cat = reverse_map.get(tech.lower())   # ищем категорию
        if cat:
            # добавляем, избегая дубликатов
            result.setdefault(cat, []).append(tech)
    
    return result


In [243]:
df_final_extracted['Technologies Categorized'] = df_final_extracted['Technologies Only'].apply(categorize)


### Final variant

In [244]:
# загружаем маппинг "lower → правильный регистр"
with open(".\\map.json", 'r', encoding='utf-8') as f:
    proper_case = json.load(f)

def fix_casing(cat_dict: dict[str, list[str]]) -> dict[str, list[str]]:
    for category, tech_list in cat_dict.items():
        cat_dict[category] = [
            proper_case.get(t.lower(), t)        # если нет в маппинге — оставляем как есть
            for t in tech_list
        ]
    return cat_dict

In [245]:
df_final_extracted['Technologies Categorized'] = df_final_extracted['Technologies Categorized'].apply(fix_casing)

## Keys to columns

In [246]:
with open(".\\key_values.json", 'r', encoding='utf-8') as f:
    data = json.load(f)

# получаем список ключей
keys_list = list(data.keys())
#keys_list

In [247]:
df_final_extracted.copy()

# -------------------------------------------------------
# 2) Гарантируем, что в колонке dict, а не строка JSON
# -------------------------------------------------------
def to_dict(x):
    """str → dict  |  dict → dict  |  NaN/пусто → {}"""
    if pd.isna(x) or (isinstance(x, str) and not x.strip()):
        return {}
    if isinstance(x, str):
        return json.loads(x)
    return x

df_final_extracted['Tech_dict'] = df_final_extracted['Technologies Categorized'].apply(to_dict)

In [248]:
# -------------------------------------------------------
# 3) Находим все уникальные категории (ключи)
# -------------------------------------------------------
all_categories = set(chain.from_iterable(df_final_extracted['Tech_dict'].map(dict.keys)))

for cat in sorted(all_categories):
    df_final_extracted[cat] = df_final_extracted['Tech_dict'].apply(
        lambda d: ', '.join(d.get(cat, []))          # список → строка
                   if d.get(cat) else ''              # нет технологий → ""
    )

In [249]:
df_final_extracted

,Location,Region,EU Member,Schengen Agreement,Google Domain Type,Google Domain Used,Job Title,Company Name,Job Location,Apply Options,...,languages_and_runtimes,monitoring_analytics_and_crash_reporting,native_interop,networking_and_api,security_and_cryptography,testing_frameworks,ui_cross_platform,ui_guidelines,ui_native,version_control_and_branching
0,Austria,Europe,True,True,default,google.com,"Android Developer – Kotlin (Austria based, Hyb...",Bitcoin Devs Company,"Vienna, Austria",Jobs3,...,Kotlin,,,Retrofit,,,,,,
1,Austria,Europe,True,True,default,google.com,ios entwickler 80–100% w/m/d,CHANCENLAND VORARLBERG,"Dornbirn, Austria","IT-Career.at, STEMJOBS.AT, IT-JOBS.AT",...,,,,,,,,,,
2,Austria,Europe,True,True,default,google.com,Middle iOS developer,Processica,"Vienna, Austria",JOBITT,...,,,,,,,,,,
3,Austria,Europe,True,True,default,google.com,iOS Developer,Raiffeisen Gruppe,"Linz, Austria","Jooble, Trabajo.org - Stellenangebote, Arbeit",...,Swift,,,,,,,,,
4,Austria,Europe,True,True,default,google.com,iOS Software Engineer,Cybermoth,"Vienna, Austria","Expertini, Talent.com",...,"Java, Kotlin, Objective-C, Swift",,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2173,United States,Northern America,False,False,default,google.com,Android Developer (Kotlin) - W2,(Not Available),"Las Vegas, NV",Teal,...,Kotlin,,,REST API,,"Espresso, JUnit",,,Jetpack Compose,
2174,United States,Northern America,False,False,default,google.com,Android Dev,ManpowerGroup,"Richmond, VA","Jobs, EarnBetter, USNLX Ability Jobs - Nationa...",...,Kotlin,,,,,,,,,
2175,United States,Northern America,False,False,default,google.com,Android Developer,codeforce360,Illinois,"SmartRecruiters Job Search, Glassdoor, Dice, J...",...,"Java, JavaScript",Google Analytics,,,OAuth,,Apache Cordova,,,
2176,United States,Northern America,False,False,default,google.com,Mobile Android Developer - INTL - LATAM OR INDIA,Insight Global,"Pittsburgh, PA",Insight Global,...,,,,,,,,,,


In [250]:
#df_final_extracted.to_csv(".\\df_final.csv", index=False)

In [5]:
#!/usr/bin/env python3
"""
Сравнение списка технологий в map.json и key_values.json.

• «map» — словарь вида {"aws": "AWS", ...}.
  Используем ТОЛЬКО ключи (левую часть), т. к. именно они должны встречаться
  в группах key_values.

• «key_values» — словарь вида {"devops": ["docker", "aws", ...], ...}.
  Собираем все элементы всех групп.

Скрипт выводит:
  * сколько и какие строки встречаются только в map;
  * сколько и какие строки встречаются только в key_values;
  * итог «OK», если отличий нет.
"""

import json
from pathlib import Path
from typing import Set

# ─── 1. Укажите пути к файлам ──────────────────────────────────────────────────
MAP_PATH = Path("./map.json")
KEY_VALUES_PATH = Path("./key_values.json") 
# ───────────────────────────────────────────────────────────────────────────────

def load_map_keys(path: Path) -> Set[str]:
    with path.open(encoding="utf-8") as f:
        data = json.load(f)
    # Берём только «сырые» ключи (lower-case названия технологий)
    return {k.strip() for k in data.keys()}

def load_key_values_items(path: Path) -> Set[str]:
    with path.open(encoding="utf-8") as f:
        data = json.load(f)
    # Все элементы всех групп
    return {item.strip() for group in data.values() for item in group}

def main() -> None:
    map_set = load_map_keys(MAP_PATH)
    kv_set = load_key_values_items(KEY_VALUES_PATH)

    only_map = sorted(map_set - kv_set)
    only_kv = sorted(kv_set - map_set)

    print(f"🗺️  В map, но не в key_values: {len(only_map)}")
    if only_map:
        print(*only_map, sep="\n  • ")

    print(f"\n📂 В key_values, но не в map: {len(only_kv)}")
    if only_kv:
        print(*only_kv, sep="\n  • ")

    if not only_map and not only_kv:
        print("\n✅ Всё распределено правильно (разницы нет).")

if __name__ == "__main__":
    main()


🗺️  В map, но не в key_values: 0

📂 В key_values, но не в map: 0

✅ Всё распределено правильно (разницы нет).
